In [1]:
import os
import sys
import json
import random
import pathlib
from tqdm import tqdm
from collections import Counter

sys.path.append("/home/arnaik/OracleProject")
random.seed(42) # seed for deterministic behavior

from src.meta_linting_dataset import MetaLinterDatasetEfficient

/home/arnaik/anaconda3/envs/py3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi

No devices were found


In [3]:
cd "/home/arnaik/OracleProject"

/home/arnaik/OracleProject


In [4]:
dataset = MetaLinterDatasetEfficient("ruff", "./data/ruff_results/", max_code_lines=200)

112it [00:14,  7.56it/s]
5000it [00:01, 4422.59it/s]    | 0/129 [00:00<?, ?it/s]
100%|██████████| 5000/5000 [00:00<00:00, 55196.78it/s]
2it [00:00, 5652.70it/s]       | 1/129 [00:01<02:47,  1.31s/it]
100%|██████████| 2/2 [00:00<00:00, 29228.60it/s]
5000it [00:01, 3723.21it/s]
100%|██████████| 5000/5000 [00:00<00:00, 53530.18it/s]
5000it [00:01, 3112.63it/s]    | 3/129 [00:02<01:54,  1.10it/s]
100%|██████████| 5000/5000 [00:00<00:00, 57890.80it/s]
5000it [00:01, 3329.54it/s]    | 4/129 [00:04<02:31,  1.21s/it]
100%|██████████| 5000/5000 [00:00<00:00, 56459.90it/s]
5000it [00:01, 2805.51it/s]    | 5/129 [00:06<02:48,  1.36s/it]
100%|██████████| 5000/5000 [00:00<00:00, 56707.83it/s]
5000it [00:01, 3679.00it/s]    | 6/129 [00:08<03:11,  1.56s/it]
100%|██████████| 5000/5000 [00:00<00:00, 58359.11it/s]
5000it [00:01, 3447.01it/s]    | 7/129 [00:09<03:09,  1.55s/it]
100%|██████████| 5000/5000 [00:00<00:00, 55316.31it/s]
5000it [00:02, 2098.65it/s]    | 8/129 [00:11<03:12,  1.59s/it]
100%|████

In [5]:
def balance_neutral_and_flagged_files(
        data: list,
        neutral_file_to_flagged_file_ratio: float=1.0,
    ):
    # iterate over data and create a list of neutral files and flagged files.
    neutral_files = []
    flagged_files = []
    for rec in data:
        response = rec['messages'][1]['content']
        if response.strip() == "NO VIOLATIONS FOUND": neutral_files.append(rec)
        else: flagged_files.append(rec)
    
    # balance the amount of neutral and modified files.
    num_neutral_files = min(int(neutral_file_to_flagged_file_ratio*len(flagged_files)), len(neutral_files))
    neutral_files = random.sample(neutral_files, k=num_neutral_files)
    data = neutral_files + flagged_files 
    data = random.sample(data, k=len(data)) # shuffle the data around.

    return data
    

In [ ]:
list(dataset.linter_data.values())[1]

In [8]:
train_idiom_mix = [
    ["F405", "F501", "F502", "F601", "F621"],
    ["E402", "E701", "E721", "E741", "E743"],
    ["N801", "N802", "N803", "N804", "N805"],
    ["N806", "N807", "N811", "N812", "N813"],
    ["UP001", "UP003", "UP004", "UP005", "UP006"],
    ["UP007", "UP008", "UP009", "UP010", "UP011"],
    ["UP044", "UP045", "UP046", "UP047", "UP040"],
    ["ERA001", "C901", "I001", "I002", "BLE001"],
    ["B002", "B003", "B004", "B005", "B006"],
    ["B007", "B008", "B009", "B010", "B012"],
]
test_idiom_mix = [
    ["F406", "F403", "F503", "F602", "F622"],
    ["E401", "E702", "E722", "E731", "E742"],
    ["ERA001", "C901", "I001", "I002", "BLE001"],
    ["ANN001", "ANN002", "ANN003", "ANN201", "ANN202"],
    ["ASYNC100", "ASYNC105", "ASYNC109", "ASYNC110", "ASYNC115"],
    ["ASYNC116", "ASYNC210", "ASYNC220", "ASYNC221", "ASYNC222"],
    ["ASYNC230", "ASYNC251", "ANN204", "ANN205", "ANN206"],
    ["S102", "S103", "S104", "S105", "S106"],
    ["S107", "S108", "S110", "S112", "S113"],
    ["S201", "S202", "S301", "S302", "S303"],
]

In [ ]:
all_train_data = []
all_test_data = []
random.seed(42)

for idiom_mix in train_idiom_mix:
    mix_data = dataset.generate_data_mix(idiom_mix, max_code_lines=200)
    print(idiom_mix, len([rec for rec in mix_data if rec['messages'][1]['content'] != 'NO VIOLATIONS FOUND']))
    all_train_data.extend(mix_data)
    
for idiom_mix in test_idiom_mix:
    mix_data = dataset.generate_data_mix(idiom_mix, max_code_lines=200)
    print(idiom_mix, len([rec for rec in mix_data if rec['messages'][1]['content'] != 'NO VIOLATIONS FOUND']))
    all_test_data.extend(mix_data)
    
print(len(all_train_data))
print(len(all_test_data))

In [ ]:
!ls

In [ ]:
# mix_data = dataset.generate_data_mix(['ERA001'])
# print(len(mix_data))
# mix_data[2]['messages'][1]['content']
# print(len([rec for rec in mix_data if rec['messages'][1]['content'] != 'NO VIOLATIONS FOUND']))

In [ ]:
random.seed(42)
from collections import defaultdict

def balance_neutral_and_flagged_files(
        data: list,
        neutral_file_to_flagged_file_ratio: float=1.0,
    ):
    # iterate over data and create a list of neutral files and flagged files.
    neutral_files = []
    flagged_files = []
    for rec in data:
        response = rec['messages'][1]['content']
        if response.strip() == "NO VIOLATIONS FOUND": neutral_files.append(rec)
        else: flagged_files.append(rec)
    
    # balance the amount of neutral and modified files.
    num_neutral_files = min(int(neutral_file_to_flagged_file_ratio*len(flagged_files)), len(neutral_files))
    neutral_files = random.sample(neutral_files, k=num_neutral_files)
    data = neutral_files + flagged_files 
    data = random.sample(data, k=len(data)) # shuffle the data around.

    return data

def impose_idiom_mix_ceilings(data, ceiling: int=5000):
    """reduce size of data stratified by the idiom mix and violation or no violation category"""
    category_to_data = defaultdict(lambda: [])
    for rec in data:
        violation_present = "yes" if rec['messages'][1]['content'] != "NO VIOLATIONS FOUND" else "no"
        category_to_data[rec['source']+"-"+violation_present].append(rec)
    category_to_data = dict(category_to_data)
    final_data = []
    for category, data_subset in category_to_data.items():
        selected_data = random.sample(data_subset, k=min(len(data_subset), ceiling))
        final_data.extend(selected_data)
        # print(category, len(selected_data))
    # print(len(final_data))
    return final_data

def split_train_and_test_data(train_data, test_data):
    train_ids = set()
    test_ids = set()
    id_to_data = {}

    for rec in train_data:
        train_ids.add(rec['id'])
        id_to_data[rec['id']] = rec
    for rec in test_data:
        test_ids.add(rec['id'])
        id_to_data[rec['id']] = rec
    
    common_ids = train_ids.intersection(test_ids)
    train_only_ids = train_ids.difference(test_ids)
    test_only_ids = test_ids.difference(train_ids)

    train_only_data = impose_idiom_mix_ceilings([id_to_data[ID] for ID in train_only_ids], ceiling=5000)
    test_only_data = impose_idiom_mix_ceilings([id_to_data[ID] for ID in test_only_ids], ceiling=500)
    # common_data = [id_to_data[ID] for ID in test_only_ids]
    common_data_split_1_IDs = set(random.sample(list(common_ids), k=len(common_ids)//2)) 
    common_data_split_2_IDs = common_ids.difference(common_data_split_1_IDs)
    common_data_split_1 = [id_to_data[ID] for ID in common_data_split_1_IDs]
    common_data_split_2 = [id_to_data[ID] for ID in common_data_split_2_IDs]
    
    train_from_common_data = impose_idiom_mix_ceilings(common_data_split_1, ceiling=5000)
    test_from_common_data = impose_idiom_mix_ceilings(common_data_split_2, ceiling=500)

    print(len(train_from_common_data))
    print(len(test_from_common_data))
    print(len(train_only_data))
    print(len(test_only_data))

    return train_only_data+train_from_common_data, test_only_data+test_from_common_data

# filt_train_data, filt_test_data = split_train_and_test_data(all_train_data, all_test_data)

In [ ]:
# print(filt_train_data[0]['messages'][0]['content'])
print(shuffle_data(filt_train_data)[0]['messages'][1]['content'])

In [6]:
# with open("./data/ruff_meta_linting/train_v3.json", "w") as f:
#     print(f"train data len: {len(filt_train_data)}")
#     # print(dict(Counter([rec['source'] for rec in filt_train_data if rec['messages'][1]['content'] != 'NO VIOLATIONS FOUND']).most_common()))
#     # print(dict(Counter([rec['source'] for rec in filt_train_data if rec['messages'][1]['content'] == 'NO VIOLATIONS FOUND']).most_common()))
#     print(dict(Counter([rec['source'] for rec in filt_train_data]).most_common()))
#     json.dump(shuffle_data(filt_train_data), f, indent=4)
# with open("./data/ruff_meta_linting/test_v3.json", "w") as f:
#     print(f"test data len: {len(filt_test_data)}")
#     # print(dict(Counter([rec['source'] for rec in filt_test_data if rec['messages'][1]['content'] != 'NO VIOLATIONS FOUND']).most_common()))
#     # print(dict(Counter([rec['source'] for rec in filt_test_data if rec['messages'][1]['content'] == 'NO VIOLATIONS FOUND']).most_common()))
#     print(dict(Counter([rec['source'] for rec in filt_test_data]).most_common()))
#     json.dump(shuffle_data(filt_test_data), f, indent=4)

## Idiom Learnability/Hardness Experiment Data Creation
Dataset with the same idioms in the train and test split (and same idiom mix as the original test set) to evaluate 3 things:
1. is the task learnable/doable
2. is the training framework working properly
3. are some idioms harder to learn than others? and which one (also is it related to training frequency).

In [13]:
random.seed(42)
from collections import defaultdict

def load_linter_results(text):
    results = []
    idiom_code = None
    for line in text.split("\n"):
        line = line.strip()
        if line == "": continue
        elif line.startswith("**Idiom") and line.endswith("Violations:**"):
            idiom_code = line.removesuffix("Violations:**").removeprefix("**Idiom").strip()
        elif line == "NO VIOLATIONS FOUND": continue
        else:
            try: 
                result = json.loads(line)
                result["code"] = idiom_code
                results.append(result)
            except Exception as e: pass
                # print(e)
                # print(f"{e}: {line}")
    return results

def create_random_split_of_given_size(data, split_1_size: int):
    split_1, split_2 = [], []
    split_1_indices = random.sample(range(len(data)), k=split_1_size)
    put_in_split_1 = [False for _ in range(len(data))]
    for index in split_1_indices:
        put_in_split_1[index] = True
    for index in range(len(data)):
        if put_in_split_1[index]:
            split_1.append(data[index])
        else: split_2.append(data[index])

    return split_1, split_2

def impose_idiom_mix_ceilings_and_split_data(data, train_ceiling: int=5000, test_ceiling: int=500):
    """reduce size of data stratified by the idiom mix and violation or no violation category"""
    category_to_data = defaultdict(lambda: [])
    violation_present_counts = defaultdict(lambda: 0) # the counts of data with at least one violation present for the meta-task idioms.
    for rec in data:
        violation_present = "yes" if load_linter_results(rec['messages'][1]['content']) != [] else "no"
        category_to_data[rec['source']+"-"+violation_present].append(rec)
    for category, data_subset in category_to_data.items():
        if category.endswith("-yes"):
            violation_present_counts[category.replace("-yes","")] = len(data_subset)

    category_to_data = dict(category_to_data)
    final_train_data = []
    final_test_data = []
    for category, data_subset in category_to_data.items():
        subset_max_possible_size = violation_present_counts[category.replace("-yes","").replace("-no","")]
        # selected_data = random.sample(data_subset, k=min(len(data_subset), train_ceiling+test_ceiling))
        selected_data = random.sample(data_subset, k=min(subset_max_possible_size, train_ceiling+test_ceiling))
        data_selected_for_train, data_selected_for_test = create_random_split_of_given_size(
            selected_data, split_1_size=min(
                train_ceiling, 
                int(
                    train_ceiling * len(selected_data) / (train_ceiling + test_ceiling)
                )
            )
        )
        final_train_data.extend(data_selected_for_train)
        final_test_data.extend(data_selected_for_test)
        # print(category, len(selected_data))
    # print(len(final_data))
    return final_train_data, final_test_data

def impose_idiom_mix_ceilings(data, ceiling: int=5000):
    """reduce size of data stratified by the idiom mix and violation or no violation category"""
    category_to_data = defaultdict(lambda: [])
    violation_present_counts = defaultdict(lambda: 0) # the counts of data with at least one violation present for the meta-task idioms.
    for rec in data:
        violation_present = "yes" if rec['messages'][1]['content'] != "NO VIOLATIONS FOUND" else "no"
        category_to_data[rec['source']+"-"+violation_present].append(rec)
    for category, data_subset in category_to_data.items():
        if category.endswith("-yes"):
            violation_present_counts[category.replace("-yes","")] = len(data_subset)

    category_to_data = dict(category_to_data)
    final_data = []
    for category, data_subset in category_to_data.items():
        subset_max_possible_size = violation_present_counts[category.replace("-yes","").replace("-no","")]
        # selected_data = random.sample(data_subset, k=min(len(data_subset), train_ceiling+test_ceiling))
        selected_data = random.sample(data_subset, k=min(subset_max_possible_size, ceiling))
        final_data.extend(selected_data)
        # print(category, len(selected_data))
    # print(len(final_data))
    return final_data

def shuffle_data(data: list[dict]):
    return random.sample(data, k=len(data))

# filt_train_data, filt_test_data = impose_idiom_mix_ceilings_and_split_data(all_data)

In [6]:
dataset.code_idiom_specs = dataset.load_ruff_idiom_specs("./data/ruff_pages_new")
print(len(dataset.code_idiom_specs))

920


In [9]:
train_data[0]

{'id': 'F401-F402-F403-F404-F405_786a044d698977dfd45eb5ab5cd4ec03824e21fa',
 'messages': [{'content': 'Look at the following list of code idiom specifications with definitions and examples:\n# Idiom F401 (unused-import)\n\nDefinition: Checks for unused imports.\n\nRationale: Unused imports add a performance overhead at runtime, and risk creating\nimport cycles. They also increase the cognitive load of reading the code.\n\n\nIf an import statement is used to check for the availability or existence\nof a module, consider using importlib.util.find_spec instead.\n\n\nIf an import statement is used to re-export a symbol as part of a module\'s\npublic interface, consider using a "redundant" import alias, which\ninstructs Ruff (and other tools) to respect the re-export, and avoid\nmarking it as unused, as in:\n\n\nfrom module import member as member\n\n\n\nAlternatively, you can use __all__ to declare a symbol as part of the module\'s\ninterface, as in:\n\n\n# __init__.py\nimport some_module\

In [14]:
# idiom_mixes = [
#     ["F406", "F403", "F503", "F602", "F622"],
#     ["E401", "E702", "E722", "E731", "E742"],
#     ["ERA001", "C901", "I001", "I002", "BLE001"],
#     ["ANN001", "ANN002", "ANN003", "ANN201", "ANN202"],
#     ["ASYNC100", "ASYNC105", "ASYNC109", "ASYNC110", "ASYNC115"],
#     ["ASYNC116", "ASYNC210", "ASYNC220", "ASYNC221", "ASYNC222"],
#     ["ASYNC230", "ASYNC251", "ANN204", "ANN205", "ANN206"],
#     ["S102", "S103", "S104", "S105", "S106"],
#     ["S107", "S108", "S110", "S112", "S113"],
#     ["S201", "S202", "S301", "S302", "S303"],
# ]
# IDIOM MIX SPANNING ALL IDIOMS:
idiom_mixes = [
    # Pyflakes
    ["F401", "F402", "F403", "F404", "F405"],
    ["F406", "F407", "F501", "F502", "F503"],
    ["F504", "F505", "F506", "F507", "F508"],
    ["F509", "F521", "F522", "F523", "F524"],
    ["F525", "F541", "F601", "F602", "F621"],
    ["F622", "F631", "F632", "F633", "F634"],
    ["F701", "F702", "F704", "F706", "F707"],
    ["F722", "F811", "F821", "F822"],
    ["F823", "F841", "F842", "F901"],
    # pygrep-hooks
    ["PGH003","PGH004","PGH005"],
    # pydoclint
    ["DOC201", "DOC202", "DOC402"],
    ["DOC403", "DOC501", "DOC502"],
    # perflint
    ["PERF101", "PERF102", "PERF203"],
    ["PERF401", "PERF402", "PERF403"],
    # pep8-naming
    ["N801", "N802", "N803", "N804"],
    ["N805", "N806", "N807", "N811"],
    ["N812", "N813", "N814", "N815"],
    ["N816", "N817", "N818", "N999"],
    # pycodestyle (E501 - line too long is skipped)
    ["E101", "E111", "E112", "E113", "E114"],
    ["E115", "E116", "E117", "E201", "E202"],
    ["E203", "E204", "E211", "E221", "E222"],
    ["E223", "E224", "E225", "E226", "E227"],
    ["E228", "E231", "E241", "E242", "E251"],
    ["E252", "E261", "E262", "E265", "E266"],
    ["E271", "E272", "E273", "E274", "E275"],
    ["E301", "E302", "E303", "E304", "E305"],
    ["E306", "E401", "E402", "E502"],
    ["E701", "E702", "E703", "E711", "E713"],
    ["E714", "E721", "E722", "E731"],
    ["E741", "E742", "E743", "E902"],
    ["W291", "W292", "W293"],
    ["W391", "W505", "W605"],
    # pathlib + flynt (FLY)
    ["PTH100", "PTH101", "PTH102", "PTH103", "PTH104"],
    ["PTH105", "PTH106", "PTH107", "PTH108", "PTH109"],
    ["PTH110", "PTH111", "PTH112", "PTH113", "PTH114"],
    ["PTH115", "PTH116", "PTH117", "PTH118", "PTH119"],
    ["PTH120", "PTH121", "PTH122", "PTH123", "PTH124"],
    ["PTH201", "PTH202", "PTH203", "PTH204", "PTH205"],
    ["PTH206", "PTH207", "PTH208", "PTH210", 'FLY002'],
    # Misc.
    ["ERA001", "C901", "I001", "I002", "BLE001"],
    # Numpy
    ["NPY001", "NPY002", "NPY003", "NPY201"],
    # Pandas
    ["PD002", "PD003", "PD004", 'PD007', 'PD008'],
    ["PD009", "PD010", "PD011", 'PD012'],
    ['PD013', "PD015", "PD101", 'PD901'],
    # pydocstyle
    ["D100", "D101", "D102", "D103", "D104"],
    ["D105", "D106", "D107", "D200", "D201"],
    ["D202", "D203", "D204", "D205", "D206"],
    ["D207", "D208", "D209", "D210", "D211"],
    ["D212", "D213", "D214", "D215", "D300"],
    ["D301", "D400", "D401", "D402", "D403"],
    ["D404", "D405", "D406", "D407", "D408"],
    ["D409", "D410", "D411", "D412"],
    ["D413", "D414", "D415", "D416"],
    ["D417", "D418", 'D419'],
    # pylint 
    # (PLC)
    ["PLC0105", "PLC0131", "PLC0132", "PLC0205", "PLC0206"],
    ["PLC0208", "PLC0414", "PLC0415", "PLC1802", "PLC1901"],
    ["PLC2401", "PLC2403", "PLC2701", "PLC2801", "PLC3002"],
    # (PLE)
    ["PLE0100", "PLE0101", "PLE0115", "PLE0116", "PLE0117"],
    ["PLE0118", "PLE0237", "PLE0241", "PLE0302", "PLE0303"],
    ["PLE0304", "PLE0305", "PLE0307", "PLE0308", "PLE0309"],
    ["PLE0604", "PLE0605", "PLE0643", "PLE1132"],
    ["PLE1141", "PLE1142", "PLE1205", "PLE1206", "PLE1300"],
    ["PLE1307", "PLE1310", "PLE1507", "PLE1519", "PLE1520"],
    ["PLE2502", "PLE2512", "PLE2513", "PLE2514"],
    ["PLE0704", "PLE1700", "PLE2515", "PLE4703"],
    # (PLR)
    ["PLR0911", "PLR0912", "PLR0913", "PLR0914", "PLR0915"],
    ["PLR0916", "PLR0917", "PLR1702", "PLR1704", "PLR1711"],
    ["PLR1714", "PLR1716", "PLR1722", "PLR1730"],
    ["PLR1733", "PLR1736", "PLR2004", "PLR2044"],
    ["PLR5501", "PLR6104", "PLR6201", "PLR6301"],
    # (PLW)
    ["PLW0108", "PLW0120", "PLW0127", "PLW0128", "PLW0129"],
    ["PLW0131", "PLW0133", "PLW0177", "PLW0211", "PLW0244"],
    ["PLW0245", "PLW0406", "PLW0602", "PLW0603", "PLW0604"],
    ["PLW0642", "PLW0711", "PLW1501", "PLW1507", "PLW1508"],
    ["PLW1509", "PLW1510", "PLW1514", "PLW1641"], 
    ["PLW2101", "PLW2901", "PLW3201", "PLW3301"],
    # pyupgrade (UP)
    ["UP001", "UP003", "UP004", "UP005", "UP006"],
    ["UP007", "UP008", "UP009", "UP010", "UP011"],
    ["UP012", "UP013", "UP014", "UP015", "UP017"],
    ["UP018", "UP019", "UP020", "UP021", "UP022"],
    ["UP023", "UP024", "UP025", "UP026", "UP028"],
    ["UP029", "UP030", "UP031", "UP032", "UP033"],
    ["UP034", "UP035", "UP036", "UP037", "UP039"],
    ["UP040", "UP041", "UP042", "UP043", "UP044"],
    ["UP045", "UP046", "UP047", "UP049"],
    # refurb (FURB)
    ["FURB101", "FURB103", "FURB105", "FURB110", "FURB113"],
    ["FURB116", "FURB118", "FURB122", "FURB129", "FURB131"],
    ["FURB132", "FURB136", "FURB140", "FURB142", "FURB145"],
    ["FURB148", "FURB152", "FURB154", "FURB156", "FURB157"],
    ["FURB161", "FURB162", "FURB163", "FURB164"],
    ["FURB166", "FURB167", "FURB168", "FURB169"],
    ["FURB171", "FURB177", "FURB180", "FURB181"],
    ["FURB187", "FURB188", "FURB189", "FURB192"],
    # ruff specifc rules (RUF)
    ["RUF001", "RUF002", "RUF003", "RUF005", "RUF006"],
    ["RUF007", "RUF008", "RUF009", "RUF010", "RUF012"],
    ["RUF013", "RUF015", "RUF016", "RUF017", "RUF018"],
    ["RUF019", "RUF020", "RUF021", "RUF022", "RUF023"],
    ["RUF024", "RUF026", "RUF027", "RUF028", "RUF029"],
    ["RUF030", "RUF031", "RUF032", "RUF033", "RUF034"],
    ["RUF036", "RUF037", "RUF038", "RUF039", "RUF040"],
    ["RUF041", "RUF043", "RUF045", "RUF046", "RUF047"],
    ["RUF048", "RUF049", "RUF051", "RUF052", "RUF053"],
    ["RUF054", "RUF055", "RUF056", "RUF057", "RUF058"],
    ["RUF059", "RUF100", "RUF101", "RUF102", "RUF200"],
    # tryceratops (TRY)
    ["TRY002", "TRY003", "TRY004", "TRY201", "TRY203"],
    ["TRY300", "TRY301", "TRY400", "TRY401"],
    # flake8-unused-arguments (ARG)
    ["ARG001", "ARG002", "ARG003", "ARG004", "ARG005"],
    # flake8-type-checking (TC)
    ["TC001", "TC002", "TC003", "TC004", "TC005"],
    ["TC006", "TC007", "TC008", "TC010"],
    # flake8-todos (TD)
    ["TD001", "TD002", "TD003", "TD004"],
    ["TD005", "TD006", "TD007"],
    # flake8-tidy-imports (TID)
    ["TID251", "TID252", "TID253"],
    # flake8-slots (SLOT)
    ["SLOT000", "SLOT001", "SLOT002"],
    # flake8-simplify (SIM) + flake8-self (SLF) + flake8-raise (RSE)
    ["SIM101", "SIM102", "SIM103", "SIM105"],
    ["SIM107", "SIM108", "SIM109", "SIM110", "SIM112"],
    ["SIM113", "SIM114", "SIM115", "SIM116", "SIM117"],
    ["SIM118", "SIM201", "SIM202", "SIM208", "SIM210"],
    ["SIM211", "SIM212", "SIM220", "SIM221", "SIM222"],
    ["SIM223", "SIM300", "SIM401", "SLF001"],
    ["SIM905", "SIM910", "SIM911", "RSE102"],
    # flake8-return (RET)
    ["RET501", "RET502", "RET503", "RET504"],
    ["RET505", "RET506", "RET507", "RET508"],
    # flake8-quotes (Q)
    ["Q001", "Q002", "Q003", "Q004"],
    # flake8-pytest-style (PT)
    ["PT001", "PT002", "PT003", "PT006", "PT007"],
    ["PT008", "PT009", "PT010", "PT011", "PT012"],
    ["PT013", "PT014", "PT015", "PT016", "PT017"],
    ["PT018", "PT019", "PT020", "PT021", "PT022"],
    ["PT023", "PT024", "PT025", "PT026", "PT027"],
    ["PT028", "PT029", "PT030", "PT031"],
    # flake8-pyi (PYI)
    ["PYI001", "PYI002", "PYI003", "PYI004", "PYI005"],
    ["PYI006", "PYI007", "PYI008", "PYI009", "PYI010"],
    ["PYI011", "PYI012", "PYI013", "PYI014", "PYI015"],
    ["PYI016", "PYI017", "PYI018", "PYI019", "PYI020"],
    ["PYI021", "PYI024", "PYI025", "PYI026", "PYI029"],
    ["PYI030", "PYI032", "PYI033", "PYI034", "PYI035"],
    ["PYI036", "PYI041", "PYI042", "PYI043", "PYI044"],
    ["PYI045", "PYI046", "PYI047", "PYI048", "PYI049"],
    ["PYI050", "PYI051", "PYI052", "PYI053", "PYI054"],
    ["PYI055", "PYI056", "PYI057", "PYI058", "PYI059"],
    ["PYI061", "PYI062", "PYI063", "PYI064", "PYI066"],
    # flake8-print (T20) + flake8-no-pep420 (INP)
    ["T201", "T203", "INP001"],
    # flake8-pie (PIE)
    ["PIE790", "PIE794", "PIE796", "PIE800"],
    ["PIE804", "PIE807", "PIE808", "PIE810"],
    # flake8-logging-format (G)
    ["G001", "G002", "G003", "G004"],
    ["G010", "G101", "G201", "G202"],
    # flake8-logging (LOG)
    ["LOG001", "LOG002", "LOG004", "LOG007"],
    ["LOG009", "LOG014", "LOG015"],
    # flake8-import-conventions (ICN)
    ["ICN001", "ICN002", "ICN003"],
    # flake8-implicit-str-concat (ISC)
    ["ISC001", "ISC002", "ISC003"],
    # flake8-gettext (INT) + flake8-future-annotations (FA)
    ["INT001", "INT002", "INT003", "FA100", "FA102"],
    # flake8-fixme (FIX)
    ["FIX001", "FIX002", "FIX003", "FIX004"],
    # flake8-executable (EXE)
    ["EXE001", "EXE002", "EXE003", "EXE004", "EXE005"],
    # flake8-errmsg (EM) + flake8-debugger (T10) + flake8-copyright (CPY)
    ["EM101", "EM102", "EM103", "T100", "CPY001"],
    # flake8-django (DJ)
    ["DJ001", "DJ003", "DJ006", "DJ007"],
    ["DJ008", "DJ012", "DJ013"],
    # flake8-datetimez (DTZ)
    ["DTZ001", "DTZ002", "DTZ003", "DTZ004", "DTZ005"],
    ["DTZ006", "DTZ007", "DTZ011", "DTZ012", "DTZ901"],
    # airflow (AIR) + FastAPI (FAST)
    ["AIR001", "AIR002", "AIR301", "AIR302", "AIR311"], 
    ["AIR312", "FAST001", "FAST002", "FAST003"],
    # flake8-2020 (YTT)
    ["YTT101", "YTT102", "YTT103", "YTT201", "YTT202"],
    ["YTT203", "YTT204", "YTT301", "YTT302", "YTT303"],
    # flake8-annotations (ANN)
    ["ANN001", "ANN002", "ANN003", "ANN201", "ANN202"],
    ["ANN204", "ANN205", "ANN206", "ANN401"],
    # flake8-async (ASYNC)
    ["ASYNC100", "ASYNC105", "ASYNC109", "ASYNC110"],
    ["ASYNC115", "ASYNC116", "ASYNC210", "ASYNC220"], 
    ["ASYNC221", "ASYNC222", "ASYNC230", "ASYNC251"],
    # flake8-bandit (S) + flake8-boolean-trap (FBT)
    ["S101", "S102", "S103", "S104", "S105"], 
    ["S106", "S107", "S108", "S110", "S112"], 
    ["S113", "S201", "S202", "S301", "S302"], 
    ["S303", "S304", "S305", "S306", "S307"],
    ["S308", "S310", "S311", "S312", "S313"],
    ["S314", "S315", "S316", "S317", "S318"],
    ["S319", "S321", "S323", "S324", "S401"],
    ["S402", "S403", "S404", "S405", "S406"],
    ["S407", "S408", "S409", "S411", "S412"],
    ["S413", "S415", "S501", "S502", "S503"],
    ["S504", "S505", "S506", "S507", "S508"],
    ["S509", "S601", "S602", "S603", "S604"],
    ["S605", "S606", "S607", "S608", "S609"],
    ["S610", "S611", "S612", "S701", "S702"], 
    ["S704", "FBT001", "FBT002", "FBT003"],
    # flake8-bugbear (B)
    ["B002", "B003", "B004", "B005", "B006"],
    ["B007", "B008", "B009", "B010", "B011"],
    ["B012", "B013", "B014", "B015", "B016"],
    ["B017", "B018", "B019", "B020", "B021"],
    ["B022", "B023", "B024", "B025", "B026"],
    ["B027", "B028", "B029", "B030", "B031"],
    ["B032", "B033", "B034", "B035", "B039"],
    ["B901", "B903", "B904"], 
    ["B905", "B909", "B911"],
    # flake8-builtins (A)
    ["A001", "A002", "A003"],
    ["A004", "A005", "A006"],
    # flake8-commas (COM)
    ["COM812", "COM818", "COM819"],
    # flake8-comprehensions (C4)
    ["C400", "C401", "C402", "C403", "C404"],
    ["C405", "C406", "C408", "C409", "C410"],
    ["C411", "C413", "C414", "C415", "C416"],
    ["C417", "C418", "C419", "C420"],
]

train_data = []
test_data = []
random.seed(42)

for idiom_mix in tqdm(idiom_mixes):
    mix_data = dataset.generate_data_mix(idiom_mix, k=1000)
    print(idiom_mix, len([rec for rec in mix_data if rec['messages'][1]['content'] != 'NO VIOLATIONS FOUND']))
    mix_train, mix_test = impose_idiom_mix_ceilings_and_split_data(mix_data, train_ceiling=50, test_ceiling=20)
    
    train_data.extend(mix_train)
    test_data.extend(mix_test)
    del mix_data
    del mix_train
    del mix_test
    
print(len(train_data))
print(len(test_data))

  0%|          | 0/197 [00:00<?, ?it/s]

  1%|          | 1/197 [00:00<00:33,  5.80it/s]

420
920
list index out of range
list index out of range
list index out of range
['F401', 'F402', 'F403', 'F404', 'F405'] 917
22
522
['F406', 'F407', 'F501', 'F502', 'F503'] 522


  2%|▏         | 3/197 [00:00<00:20,  9.62it/s]

45
545
list index out of range
['F504', 'F505', 'F506', 'F507', 'F508'] 544
219
719
['F509', 'F521', 'F522', 'F523', 'F524'] 719
333
833
['F525', 'F541', 'F601', 'F602', 'F621'] 833


...

['COM812', 'COM818', 'COM819'] 997
497
997
['C400', 'C401', 'C402', 'C403', 'C404'] 997
434
934


 99%|█████████▉| 196/197 [00:20<00:00,  9.10it/s]

['C405', 'C406', 'C408', 'C409', 'C410'] 934
361
861
['C411', 'C413', 'C414', 'C415', 'C416'] 861
417
917
['C417', 'C418', 'C419', 'C420'] 917


100%|██████████| 197/197 [00:20<00:00,  9.47it/s]

16682
6702


In [11]:
print(train_data[0]['messages'][1]['content'])

**Idiom F401 Violations:**

{"line": " 13 import csv", "span": "csv", "fix": [{"before": "import csv\n", "after": ""}]}

**Idiom F402 Violations:**

NO VIOLATIONS FOUND

**Idiom F403 Violations:**

{"line": " 14 from csv import *", "span": "from csv import *", "fix": null}

**Idiom F404 Violations:**

NO VIOLATIONS FOUND

**Idiom F405 Violations:**

{"line": " 17 \tcsv_writer=writer(fileobj,delimiter=',')", "span": "writer", "fix": null}




In [15]:
# filt_train_data, filt_test_data = impose_idiom_mix_ceilings_and_split_data(all_data)

with open("./data/ruff_meta_linting/all_idioms/train.json", "w") as f:
    print(f"train data len: {len(train_data)}")
    print(dict(Counter([rec['source'] for rec in train_data if load_linter_results(rec['messages'][1]['content']) != []]).most_common()))
    print(dict(Counter([rec['source'] for rec in train_data if load_linter_results(rec['messages'][1]['content']) == []]).most_common()))
    print(dict(Counter([rec['source'] for rec in train_data]).most_common()))
    json.dump(shuffle_data(train_data), f, indent=4)
with open("./data/ruff_meta_linting/all_idioms/test.json", "w") as f:
    print(f"test data len: {len(test_data)}")
    print(dict(Counter([rec['source'] for rec in test_data if load_linter_results(rec['messages'][1]['content']) != []]).most_common()))
    print(dict(Counter([rec['source'] for rec in test_data if load_linter_results(rec['messages'][1]['content']) == []]).most_common()))
    print(dict(Counter([rec['source'] for rec in test_data]).most_common()))
    json.dump(shuffle_data(test_data), f, indent=4)

train data len: 16682
{'rull_linter/F401-F402-F403-F404-F405': 50, 'rull_linter/F509-F521-F522-F523-F524': 50, 'rull_linter/F525-F541-F601-F602-F621': 50, 'rull_linter/F622-F631-F632-F633-F634': 50, 'rull_linter/F701-F702-F704-F706-F707': 50, 'rull_linter/F722-F811-F821-F822': 50, 'rull_linter/F823-F841-F842-F901': 50, 'rull_linter/PGH003-PGH004-PGH005': 50, 'rull_linter/PERF101-PERF102-PERF203': 50, 'rull_linter/PERF401-PERF402-PERF403': 50, 'rull_linter/N801-N802-N803-N804': 50, 'rull_linter/N805-N806-N807-N811': 50, 'rull_linter/N812-N813-N814-N815': 50, 'rull_linter/N816-N817-N818-N999': 50, 'rull_linter/E101-E111-E112-E113-E114': 50, 'rull_linter/E306-E401-E402-E502': 50, 'rull_linter/E701-E702-E703-E711-E713': 50, 'rull_linter/E714-E721-E722-E731': 50, 'rull_linter/E741-E742-E743-E902': 50, 'rull_linter/W291-W292-W293': 50, 'rull_linter/W391-W505-W605': 50, 'rull_linter/PTH100-PTH101-PTH102-PTH103-PTH104': 50, 'rull_linter/PTH105-PTH106-PTH107-PTH108-PTH109': 50, 'rull_linter/PTH

## Idiom Transfer Learning
Dataset with the some of the same idioms, some paired near transfer idioms and some unseen test only idioms/meta-tasks to evaluate tendency for memorization vs instruction following/reasoning in LLMs.

In [ ]:
train_only_idiom_mix = [
    ["F405", "F501", "F502", "F601", "F621"],
    ["E402", "E701", "E721", "E741", "E743"],
    ["N801", "N802", "N803", "N804", "N805"],
    ["N806", "N807", "N811", "N812", "N813"],
    ["UP001", "UP003", "UP004", "UP005", "UP006"],
    ["UP007", "UP008", "UP009", "UP010", "UP011"],
    ["UP044", "UP045", "UP046", "UP047", "UP040"],
    ["B002", "B003", "B004", "B005", "B006"],
    ["B007", "B008", "B009", "B010", "B012"],
]
test_only_idiom_mix = [
    ["F406", "F403", "F503", "F602", "F622"],
    ["E401", "E702", "E722", "E731", "E742"],
    ["ANN001", "ANN002", "ANN003", "ANN201", "ANN202"],
    ["ASYNC100", "ASYNC105", "ASYNC109", "ASYNC110", "ASYNC115"],
    ["ASYNC116", "ASYNC210", "ASYNC220", "ASYNC221", "ASYNC222"],
    ["ASYNC230", "ASYNC251", "ANN204", "ANN205", "ANN206"],
    ["S102", "S103", "S104", "S105", "S106"],
    ["S107", "S108", "S110", "S112", "S113"],
    ["S201", "S202", "S301", "S302", "S303"],
]

shared_data = [] # basically no-transfer setting/
random.seed(42)

shared_idiom_mix = ["ERA001", "C901", "I001", "I002", "BLE001"]
shared_data = dataset.generate_data_mix(shared_idiom_mix, max_code_lines=200)
print(shared_idiom_mix, len([rec for rec in shared_data if rec['messages'][1]['content'] != 'NO VIOLATIONS FOUND']))
shared_train_data, shared_test_data = impose_idiom_mix_ceilings_and_split_data(shared_data)
print(len(shared_train_data))
print(len(shared_test_data))

In [7]:
del shared_data

In [ ]:
train_only_data = []
test_only_data = []

for idiom_mix in train_only_idiom_mix:
    mix_data = impose_idiom_mix_ceilings(dataset.generate_data_mix(idiom_mix, max_code_lines=200), ceiling=5000)
    print(idiom_mix, len([rec for rec in mix_data if rec['messages'][1]['content'] != 'NO VIOLATIONS FOUND']))
    train_only_data.extend(mix_data)
    del mix_data
for idiom_mix in test_only_idiom_mix:
    mix_data = impose_idiom_mix_ceilings(dataset.generate_data_mix(idiom_mix, max_code_lines=200), ceiling=500)
    print(idiom_mix, len([rec for rec in mix_data if rec['messages'][1]['content'] != 'NO VIOLATIONS FOUND']))
    test_only_data.extend(mix_data)
    del mix_data

# train_only_data = impose_idiom_mix_ceilings(all_train_data, ceiling=5000)
# del all_train_data
# test_only_data = impose_idiom_mix_ceilings(all_test_data, ceiling=500)
# del all_test_data

print(len(train_only_data))
print(len(test_only_data))

In [14]:
filt_train_data = train_only_data + shared_train_data
filt_test_data = test_only_data + shared_test_data

# # shuffle data
# filt_train_data = random.sample(filt_train_data, k=len(filt_train_data))
# filt_test_data = random.sample(filt_test_data, k=len(filt_test_data))

In [ ]:
with open("./data/ruff_meta_linting/train_v4.json", "w") as f:
    print(f"train data len: {len(filt_train_data)}")
    print(dict(Counter([rec['source'] for rec in filt_train_data if rec['messages'][1]['content'] != 'NO VIOLATIONS FOUND']).most_common()))
    print(dict(Counter([rec['source'] for rec in filt_train_data if rec['messages'][1]['content'] == 'NO VIOLATIONS FOUND']).most_common()))
    print(dict(Counter([rec['source'] for rec in filt_train_data]).most_common()))
    json.dump(shuffle_data(filt_train_data), f, indent=4)
with open("./data/ruff_meta_linting/test_v4.json", "w") as f:
    print(f"test data len: {len(filt_test_data)}")
    print(dict(Counter([rec['source'] for rec in filt_test_data if rec['messages'][1]['content'] != 'NO VIOLATIONS FOUND']).most_common()))
    print(dict(Counter([rec['source'] for rec in filt_test_data if rec['messages'][1]['content'] == 'NO VIOLATIONS FOUND']).most_common()))
    print(dict(Counter([rec['source'] for rec in filt_test_data]).most_common()))
    json.dump(shuffle_data(filt_test_data), f, indent=4)